In [12]:
import json
import numpy as np
import os
from glob import glob

# Function to load landmarks from a JSON file.
def load_landmarks(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Extract hand keypoints.
    hand_left_keypoints = []
    hand_right_keypoints = []
    
    for person in data['people']:
        left_hand = person['hand_left_keypoints_2d']
        right_hand = person['hand_right_keypoints_2d']
        
        if left_hand:
            hand_left_keypoints.append(left_hand)
        if right_hand:
            hand_right_keypoints.append(right_hand)
    
    return np.array(hand_left_keypoints), np.array(hand_right_keypoints)

# Function to create sequences from the landmark data.
def create_sequences(data, sequence_length=30):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

# Directory containing the JSON files.
json_dir = r"D:\dataset\openpose_output_val\output"  # Replace with the actual path to your directory
json_files = glob(os.path.join(json_dir, '*.json'))

# Initialize lists to hold all sequences and labels.
all_sequences = []
all_labels = []

# Process each JSON file.
for json_file in json_files:
    left_hand_data, right_hand_data = load_landmarks(json_file)
    
    # Create sequences for both left and right hand data.
    sequence_length = 30
    left_hand_sequences = create_sequences(left_hand_data, sequence_length)
    right_hand_sequences = create_sequences(right_hand_data, sequence_length)

    # Combine left and right hand sequences if both are available.
    if left_hand_sequences.shape[0] > 0 and right_hand_sequences.shape[0] > 0:
        combined_sequences = np.concatenate([left_hand_sequences, right_hand_sequences], axis=-1)
    else:
        combined_sequences = left_hand_sequences if left_hand_sequences.shape[0] > 0 else right_hand_sequences

    # Flatten the landmarks.
    if combined_sequences.shape[0] > 0:
        num_features = combined_sequences.shape[-2] * combined_sequences.shape[-1]
        flattened_sequences = combined_sequences.reshape(combined_sequences.shape[0], combined_sequences.shape[1], num_features)
        
        # Append to the list of all sequences.
        all_sequences.append(flattened_sequences)
        
        # Generate labels (assuming all sequences from the same file belong to the same class in this example).
        labels = np.array([1] * len(flattened_sequences))  # Change '1' to the actual label for the sign.
        all_labels.append(labels)

# Combine all sequences and labels.
all_sequences = np.concatenate(all_sequences, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Save the sequences and labels.
np.save(r"D:\dataset\new", all_sequences)
np.save(r"D:\dataset\new2", all_labels)

print(f"All sequences shape: {all_sequences.shape}")
print(f"All labels shape: {all_labels.shape}")


All sequences shape: (224517, 30, 126)
All labels shape: (224517,)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load the sequences and labels.
sequences = np.load(r"D:\dataset\new.npy")
labels = np.load(r"D:\dataset\new2.npy")
# Split the data into training and testing sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Define the LSTM model.
sequence_length = 30
num_features = sequences.shape[2]

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(sequence_length, num_features)),
    Dropout(0.5),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation='tanh'),
    Dense(1, activation='sigmoid')  # Change to the number of classes if more than 1.
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model.
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Save the model with a valid extension.
model.save(r"D:\dataset\new.keras")

print("Model saved successfully.")


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 367ms/step - accuracy: 0.6979 - loss: 0.6323 - val_accuracy: 1.0000 - val_loss: 0.4578
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9674 - loss: 0.4774 - val_accuracy: 1.0000 - val_loss: 0.3133
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.3520 - val_accuracy: 1.0000 - val_loss: 0.2002
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.2196 - val_accuracy: 1.0000 - val_loss: 0.1155
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.1461 - val_accuracy: 1.0000 - val_loss: 0.0626
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0855 - val_accuracy: 1.0000 - val_loss: 0.0328
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 0.0475 - val_accuracy: 1.0000 - val_loss: 0.0172
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.0224 - val_accuracy: 1.0000 - val_loss: 0.0093